<a href="https://colab.research.google.com/github/ZixinIsabelDeng/Predicting-the-Delisting-of-an-IPO-Company-Within-the-Next-Five-Quarters/blob/zixin/comp333project%E2%80%94DataExplorationAndCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting the Delisting of an IPO Company Within the Next Five Quarters**

In [ ]:
# a list of library to import
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# **Introduction**


In the dynamic and often volatile realm of the stock market, the ability to predict the delisting of companies post-IPO (Initial Public Offering) presents a critical advantage for investors, regulatory bodies, and economic analysts. This study focuses on the development and implementation of a predictive model aimed at forecasting the potential delisting of non-SPAC (Special Purpose Acquisition Company) IPO companies within five quarters following their market debut

**The significance of this prediction**

-Investor Decision-Making: Investors stand to gain crucial insights into potential risks associated with their current or future investments in IPO companies. Early prediction helps in mitigating losses and optimizing portfolio performance by avoiding companies with a higher likelihood of delisting.
Regulatory Oversight: Regulators can enhance their monitoring and oversight of newly listed companies, potentially identifying and addressing issues before they lead to delisting, thus maintaining market integrity and investor trust.
Market Stability: Understanding and predicting delisting trends can help stabilize market dynamics by providing all market participants with better data on the health and longevity of new entrants.


**Challenges in Predicting Delisting**

Predicting the delisting of non-SPAC IPO companies within such a short period poses several significant challenges:


The first challenge is data quality and scarcity. Reliable historical data on newly public companies, especially those facing delisting, is limited. We search multiple websites but most of them only provide listing IPO information. Fortunately, We are lucky to find delisting company information on Barchart.com and Alpha vantage API.  However, we don't know if those data are reliable.


The second challenges is related to external factors. The stock market is influenced by numerous, often unpredictable external factors like economic shifts, policy changes, and global events which can abruptly alter a company's trajectory. Our analysis is based on financial data but does not mention any information related to elements mentioned above.


In terms of Model Complexity and Bias, developing a model that accurately reflects the complexities of the real world without inheriting or amplifying biases present in historical data is a formidable task.Our project focus on SVM model and didn't use any hyperparameter tuning. If we are able to evaluate more on model selection and goes deeper in machine learning aspect. Our model can be more reliable,

How can we improve?

Our delisting company have SPAC companies. Those are mean to delisted, this factors will affect the reliability of our ml model

# load the data

In [ ]:
# download API and read excel file

%cd /content/
df1= pd.read_excel('delisted_companies_data - Copy.xlsx')
df2=pd.read_csv('updated_listed_new_companies_data.csv')

/content


NameError: name 'pd' is not defined

# **Data Cleaning**
**Step 1. Removing irrelevant columns **

This will related to non-times series data. we don't need information like company name, maket stock exchange place, symbol. We also don't need future data like delisting IPO date.

In [ ]:
df1.head()


,symbol,name,exchange,ipoDate,delistingDate,status,sector,numEmployees,salesQ5,salesQ4,...,opCashflowQ5,opCashflowQ4,opCashflowQ3,opCashflowQ2,opCashflowQ1,netCashflowQ5,netCashflowQ4,netCashflowQ3,netCashflowQ2,netCashflowQ1
0,AAIN,Arlington Asset Investment Corp,NYSE,2021-07-19,2024-01-30,Delisted,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AAQC,Accelerate Acquisition Corp - Class A,NYSE,2021-05-10,2022-12-15,Delisted,NaN,NaN,0,0,...,-850,-560,-290,-1500,-1260,-360,-280,-290,940,1170
2,ABGI,ABG Acquisition Corp I - Class A,NASDAQ,2021-02-17,2023-02-27,Delisted,NaN,NaN,0,0,...,-370,-270,-190,-980,-870,-410,-310,-240,450,560
3,ACACU,PLAYSTUDIOS Inc - Units (1 Ord Share Class A &...,NASDAQ,2020-10-23,2021-06-21,Delisted,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ACAMU,CarLotz Inc - Unit (1 Ordinary Class A & 0.33 ...,NASDAQ,2019-02-22,2021-01-21,Delisted,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#drop column sector and nonemployees due to null data, name and symnbol due to irrelevent to the prediction task

df1 = df1.drop(columns=['sector', 'numEmployees', 'exchange', 'name', 'ipoDate', 'symbol', 'delistingDate'])
df2 = df2.drop(columns=['sector', 'numEmployees', 'exchange', 'name', 'symbol', 'ipoDate'])
df2['status']='listed'

*Step 2: Fix structural errors*
In our webscrapping data set, the null data is replaced by 0 and noan. and we don't want that 0 to affect the accurancy of our model. so when change all of them to null

In [ ]:
df2.replace('None',0, inplace=True)
df1.replace('None',0, inplace=True)
df1= df1.replace(np.nan,0)
df2= df2.replace(np.nan,0)

In [ ]:
df['status']

KeyError: 'status'

Step 3. perform an initial SVM result evaluation to give a starting point for comparation. later, we will see how this result improves.

In [ ]:
data=pd.concat([df1,df2]).copy()

In [ ]:


def machine_learning_check1(data, target, test_size=0.3):
    # Exclude the 'ipoDate' column

    # we need an encoding for ml model
    data['status']=data['status'].map({'Delisted':1,'listed':0})


    # Split the training and testing set
    X = data.drop(columns=[target])
    y = data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=43)

    # Define the model using XGBoost
    pipeline = Pipeline([

        ('classifier', SVC(kernel='rbf'))  # Using RBF kernel
    ])

    # Fit the pipeline
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"SVM Accuracy = {accuracy:.4f}")




step 4: We don't need rows that has more than 50 percent of null values. Rows with a high number of missing values can significantly distort the outcomes of our analysis. If a substantial portion of the data is missing, any calculations or predictions made using this data are likely to be unreliable. By removing such rows, we ensure that the remaining dataset maintains a higher standard of data integrity and quality.

In [ ]:
#delete the whole row if all the financial data are all null for df2 or zero for df,
def create_mask(df_name, selected_data):
    if df_name == 'df1':
        mask_zero_or_null = (selected_data == 0).all(axis=1)
    elif df_name == 'df2':
        mask_zero_or_null = selected_data.isnull().all(axis=1)
    else:
        raise ValueError("DataFrame name not recognized.")
    return mask_zero_or_null


def delete_zero_or_null(df_name,start_col,df_name_string):
  start_index=df_name.columns.get_loc(start_col)
  selected_data=df_name.iloc[:,start_index:]
  mask_zero_or_null=create_mask(df_name_string, selected_data)
  return df_name[~mask_zero_or_null]


df1=delete_zero_or_null(df1,'salesQ1','df1')
df2=delete_zero_or_null(df2,'salesQ5','df2')

In [ ]:
#concate listed and delisted company
df=pd.concat([df1,df2])

In [ ]:
df['status']

In [ ]:
#some missing value shown as 0, we convert it to null value first then filling them together
def convert_zero(df_data,from_col,to_col):
  f=df_data.columns.get_loc(from_col)
  t=df_data.columns.get_loc(to_col)
  selected=df_data.iloc[:,f:t+1]
  df_data.iloc[:, f:t+1] = selected.replace(0, np.nan)
  return df
df=convert_zero(df,'salesQ5','netCashflowQ1')

In [ ]:
#most of the null value coming from sales data
(df.isnull().sum()/len(df)).sort_values(ascending=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
encoder = LabelEncoder()

# Fit and transform the 'status' column
#df['status'] = encoder.fit_transform(df['status'])
df['status']

In [ ]:
#Perform sensistive analysis to determine which null data filling technique gives the best result




def sensitivity_analysis(data,target,test_size=0.3):

  #split training and testing test
  X=data.drop(columns=target)

  y=data[target]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=43)

  #include three types of ways we wanna fill the null data
  imputers={
      'mean_imputation':SimpleImputer(strategy='mean'),
      'median_imputation':SimpleImputer(strategy='median'),
      'knn_imputation': KNNImputer(n_neighbors=5)
  }
  models = {
        'Logistic Regression': LogisticRegression(solver='liblinear',random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100,random_state=42),
        'SVM': SVC(kernel='linear',random_state=42)
    }

  results = {model_name: {} for model_name in models}
  imputer_average = {imputer_name: [] for imputer_name in imputers}

  for model_name, model in models.items():
    for imputer_name, imputer in imputers.items():
      pipeline = Pipeline([
                        ('scaler', StandardScaler()),
                        ('imputer', imputer),
                        ('classifier', model)
      ])
      pipeline.fit(X_train, y_train)
      y_pred = pipeline.predict(X_test)
      accuracy = accuracy_score(y_test, y_pred)
      results[model_name][imputer_name] = accuracy
      imputer_average[imputer_name].append(accuracy)

  for model_name, imputers in results.items():
    print(f"Model: {model_name}")
    for imputer_name, acc in imputers.items():
      print(f"  {imputer_name}: Accuracy = {acc:.4f}")
      print("\n")


  # Print average accuracies for each imputer
  print("Average Accuracies for Each Imputer Across Models:")
  for imputer_name, accuracies in imputer_average.items():
    average_accuracy = np.mean(accuracies)
    print(f"  {imputer_name}: Average Accuracy = {average_accuracy:.4f}")




sensitivity_analysis(df,'status',)




In [ ]:
#we are not able to perform filling missing value on object data, so we convert all of them to numeric data
object_cols = df.select_dtypes(include=['object']).columns
df[object_cols] = df[object_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
status=df['status'].copy()

In [ ]:
df=df.drop(columns='status')

In [ ]:
# as per result,  we select mean_imputation
for i in df.columns:
  mean=df[i].mean()
  df[i]=df[i].fillna(mean)


*Step 5: Filter out data outliers*

we are using z-score to filter out outliers

The z-score of an observation quantifies how many standard deviations the observation is from the mean of the dataset. It is calculated using the formula:

𝑧
=
(
𝑋
−
𝜇
)
𝜎


Where:

𝑋 is the value of the observation.
𝜇 is the mean of the dataset.
𝜎 is the standard deviation of the dataset.


we remove rows from a DataFrame where any numeric column has a Z-score greater than threshold=3, effectively filtering out statistical outliers. It returns a new DataFrame without these outlier rows,

In [ ]:
from scipy import stats

def remove_outliers_zscore(df, threshold=3):
    """
    Removes rows from the DataFrame where any numeric column's value has a Z-score > threshold.

    Parameters:
    - df: pandas DataFrame from which to remove outliers
    - threshold: Z-score value above which a data point is considered an outlier (default: 3)

    Returns:
    - A new DataFrame with outliers removed.
    """
    outlier_mask = np.zeros(len(df), dtype=bool)  # Initialize a mask for rows to remove

    col=['status']
    df[col]=df[col].astype(dtype='object')
    for column in df.select_dtypes(include=[np.number]).columns:
        # Calculate Z-scores for the column
        zs = np.abs(stats.zscore(df[column], nan_policy='omit'))
        # Update the mask to include outliers in the current column
        outlier_mask |= (zs > threshold)
    return df[~outlier_mask]


df = remove_outliers_zscore(df)




*step 6: a final formatting*
we make the datatype consistent and perform a ml result check to see how we improve the result

In [ ]:
df.isna().sum()

status                3035
salesQ5                  0
salesQ4                  0
salesQ3                  0
salesQ2                  0
salesQ1                  0
netIncomeQ5              0
netIncomeQ4              0
netIncomeQ3              0
netIncomeQ2              0
netIncomeQ1              0
totalAssetQ5             0
totalAssetQ4             0
totalAssetQ3             0
totalAssetQ2             0
totalAssetQ1             0
totalLiabilitiesQ5       0
totalLiabilitiesQ4       0
totalLiabilitiesQ3       0
totalLiabilitiesQ2       0
totalLiabilitiesQ1       0
opCashflowQ5             0
opCashflowQ4             0
opCashflowQ3             0
opCashflowQ2             0
opCashflowQ1             0
netCashflowQ5            0
netCashflowQ4            0
netCashflowQ3            0
netCashflowQ2            0
netCashflowQ1            0
dtype: int64

In [ ]:
df['status']

1       NaN
2       NaN
10      NaN
13      NaN
17      NaN
       ... 
2301    NaN
2302    NaN
2303    NaN
2304    NaN
2305    NaN
Name: status, Length: 3035, dtype: object

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
encoder = LabelEncoder()

# Fit and transform the 'status' column
df['status'] = encoder.fit_transform(df['status'])

<ipython-input-200-49556ad9b4fe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status'] = encoder.fit_transform(df['status'])


In [ ]:
df['status']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3030   NaN
3031   NaN
3032   NaN
3033   NaN
3034   NaN
Name: status, Length: 3035, dtype: float64

In [ ]:
df.reset_index(drop=True,inplace=True)
#change all of them to int
# Convert DataFrame to nullable integer type



machine_learning_check1(df, 'status', test_size=0.3)

ValueError: Input y contains NaN.

# **Data Transformation**

This part of data tranformation includes data normalization and apply of three different scalar to compare machine learning result on encoded dataset.

In [ ]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()

# Separate the 'status' column
status = df['status'].copy()
df_numeric = df.drop(columns='status')

# Scale the numerical data
normalized_data = normalizer.fit_transform(df_numeric)
df_scaled = pd.DataFrame(normalized_data , columns=df_numeric.columns)

# Add the 'status' column back
df_scaled['status'] = status



In [ ]:

def machine_learning_check2(data, target, test_size=0.3):
    """
    Trains and evaluates an SVM classifier with different scaling methods on the provided dataset.

    This function splits the data into training and testing sets, then applies different scalers (MinMaxScaler, StandardScaler, and RobustScaler)
    to the training data. It then trains an SVM classifier with an RBF kernel for each scaled version of the data. It evaluates each model
    on the testing set and prints the accuracy for each scaler.

    Parameters:
    - data (pandas.DataFrame): The dataset containing features and the target variable.
    - target (str): The name of the target column in `data` which should be predicted.
    - test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.3.

    Returns:
    - None: This function does not return anything but prints the accuracy of the model for each scaling method.
    """
    # Exclude the target column
    X = data.drop(columns=[target])
    y = data[target]

    # Split the training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=232)

    # Scalers to be tested
    scalers = {
        'MinMaxScaler': MinMaxScaler(),
        'StandardScaler': StandardScaler(),
        'RobustScaler': RobustScaler()
    }

    # Loop through each scaler and test it
    for scaler_name, scaler in scalers.items():
        # Define the pipeline with scaling and the SVM classifier
        pipeline = Pipeline([
            ('scaler', scaler),  # Dynamic scaler from the dictionary
            ('classifier', SVC(kernel='rbf'))  # Using RBF kernel
        ])

        # Fit the pipeline
        pipeline.fit(X_train, y_train)

        # Make predictions
        y_pred = pipeline.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{scaler_name} SVM Accuracy = {accuracy:.4f}")



machine_learning_check2(df_scaled, 'status', test_size=0.3)


data scaling: we select standard scaler

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Scale the DataFrame without the 'status' column
scaled_data = scaler.fit_transform(df_scaled)
df_scaled = pd.DataFrame(scaled_data , columns=df_scaled.columns)


In [ ]:
df_scaled['status'] = status

perform label encoding to scaled data and records ml scores

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
encoder = LabelEncoder()

# Fit and transform the 'status' column
df_scaled['status'] = encoder.fit_transform(df['status'])


In [ ]:
machine_learning_check2(df_scaled, 'status', test_size=0.3)

# **Dimensionality Reduction**
This part includes:

*features selection*

through correlation matric by selecting 0.85 as our threshold.

*features transformation*

through PCA by setting up the number of principal components as 18 as it can describe our data within 95% of variance by reducing more than 15 columns


In [ ]:
correlation_matrix = df_scaled.corr()

import seaborn as sns
import matplotlib.pyplot as plt

# checking the correlation matrix
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix of Features')
plt.show()

This part we wants to decide which columns to drop

In [ ]:
threshold = 0.85

#Find and remove the highly correlated features
to_drop = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            col_name = correlation_matrix.columns[i]
            to_drop.add(col_name)

print(f"Columns to drop: {to_drop}")

In [ ]:
# we compare SVM to check how much the result improves
#before dropping

machine_learning_check2(df_scaled, 'status', test_size=0.3)

In [ ]:
df_scaled=df_scaled.drop(columns=['salesQ4', 'salesQ5', 'opCashflowQ3', 'totalAssetQ4'])

In [ ]:
# after dropping
machine_learning_check2(df_scaled, 'status', test_size=0.3)


*features transformation*

since our ml accurancy does not improve too much. we decide to add anothe layer: PCA to captures 95% of variance and reducing the noices. we assume it will give us a better ml result

In [ ]:

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df_scaled)

# Get explained variance
explained_variances = pca.explained_variance_ratio_

how to select N principal component?   


Initially, we decide to use elbow point to decide, however, elbow points only covers 50 percent of variance. this result if not idea. then we decide to first set an idea vairance coverage, then find the amount of PCA to achieve this result.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(explained_variances) + 1), explained_variances, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

# Initialize PCA with 18 components
pca = PCA(n_components=18)

# Fit PCA on the scaled data and transform it
principalComponents = pca.fit_transform(df_scaled)

# Create a DataFrame for the principal components
principal_df = pd.DataFrame(
    data=principalComponents,
    columns=['PCA1', 'PCA2', 'PCA3', 'PCA4','PCA5','PCA6','PCA7','PCA8','PCA9','PCA10','PCA11','PCA12','PCA13','PCA14','PCA15','PCA16','PCA17','PCA18']
)




showing data after PCA tranformtion

In [ ]:
principal_df.head()

In [ ]:
# Display the DataFrame with principal components

principal_df['status']=status


# Data Exploration

In [ ]:

# we deliver this result to front end developer to perform dashboard visualization
principal_df.to_csv('principal_componet.csv')